# OmicsIntegrator

In [3]:
from MicrobeRX.Predictor.Visualizer import plot_relationships
from MicrobeRX.OmicsIntegrator import plot_species_sunburst

import pandas as pd

ModuleNotFoundError: No module named 'MicrobeRX.OmicsIntegrator'

In [ ]:
evidences=pd.read_csv('test/prednisone_evidences.tsv',sep='\t')
evidences

In [ ]:
plot_relationships(evidences,nodes=['reaction_id','origin','name','ec','metabolite_id'])

In [ ]:
plot_species_sunburst(sources=['ACCOACORAT'])

In [ ]:
from package.MicrobeRX.DataFiles import load_microbes_reactions 

In [ ]:
reactions=load_microbes_reactions()
reactions

In [ ]:
reactions[['ACCOACORAT']].dropna().to_csv('test/ACCOACORAT_ids.tsv',sep='\t',index=True)

In [ ]:
import pandas as pd

from Bio import Entrez
from Bio.Align.Applications import ClustalOmegaCommandline
from Bio import Phylo
from Bio import AlignIO, SeqIO, SeqRecord

from io import StringIO


from ete3 import PhyloTree, SeqMotifFace, TreeStyle, AttrFace, NodeStyle, faces, TextFace, Tree

Entrez.email = "a.j.ruiz.moreno@umcg.nl"

import re

from tqdm.notebook import tqdm

pattern = r"\[(.*?)\]"

In [ ]:
evidences=pd.read_csv('../../../../MicrobeRX/test/ACCOACORAT_ids.tsv',sep='\t',index_col=[0])
evidences

In [ ]:
with open('sequences.faa','w') as fasta:
    for entry in evidences.ACCOACORAT.unique():
        try:
            handle = Entrez.efetch(db="protein", id=entry, rettype="fasta", retmode="text")
            fasta.write(handle.read())
            handle.close()
        except Exception:
            pass

In [ ]:
records={r.id:r for r in SeqIO.parse('sequences.faa','fasta')}   

In [ ]:
sequences={}
for index in evidences.index:
    if evidences.ACCOACORAT[index] in records.keys():
        seq=copy.deepcopy(records[evidences.ACCOACORAT[index]])
        seq.id=index
        seq.name=index
        seq.description=evidences.ACCOACORAT[index]
        sequences[index]=seq
    else:
        pass

In [ ]:
with open("ACCOACORAT.faa", "w") as output_handle:
    SeqIO.write(sequences.values(), output_handle, "fasta")

In [ ]:
clustalo = './clustalo4'

clustalomega_cline = ClustalOmegaCommandline(cmd=clustalo,infile=f'sequences.faa', 
                                                 guidetree_out=f'ACCOACORAT.dnd',
                                                 outfile=f'ACCOACORAT.aln',
                                                 outfmt='fa',
                                                 seqtype='Protein',
                                                 verbose=False,auto=True,force=True)

In [ ]:
clustalomega_cline()

In [ ]:
aligment = SeqIO.to_dict(SeqIO.parse("ACCOACORAT.aln", "fasta"))

In [ ]:
tree = Tree(newick='ACCOACORAT.dnd',)

nstyle = NodeStyle()
nstyle["shape"] = "sphere"
nstyle["fgcolor"] = "black"
nstyle["size"] = 5

for n in tree.traverse():
    n.set_style(nstyle)

for leaf in tree.get_leaves():
    if leaf.name in ['WP_034512663.1','WP_005307587.1','WP_001112110.1']:
        seqFace = SeqMotifFace(str(aligment[leaf.name].seq), gapcolor="Gray",seq_format="compactseq",fgcolor='white',bgcolor='lightblue',height=7)
        leaf.add_face(seqFace, column=0,position= "aligned")

        txtFace=TextFace(f"  {str(aligment[leaf.name].name)}",fsize=7,bold=True,fgcolor='red')
        leaf.add_face(txtFace, column=0,position= "branch-right")

        txtFace=TextFace(f"  {str(aligment[leaf.name].description)}",fsize=7,bold=False,fgcolor='red')
        leaf.add_face(txtFace, column=1,position= "aligned")
    
    else:
        
        seqFace = SeqMotifFace(str(aligment[leaf.name].seq), gapcolor="Gray",seq_format="compactseq",fgcolor='white',bgcolor='lightblue',height=7)
        leaf.add_face(seqFace, column=0,position= "aligned")

        txtFace=TextFace(f"  {str(aligment[leaf.name].name)}",fsize=7,bold=False)
        leaf.add_face(txtFace, column=0,position= "branch-right")

        txtFace=TextFace(f"  {str(aligment[leaf.name].description)}",fsize=7,bold=False)
        leaf.add_face(txtFace, column=1,position= "aligned")
    

ts = TreeStyle()
ts.show_leaf_name = False
ts.show_branch_length = False
ts.show_branch_support=False
ts.scale = 10
ts.mode = "r"
ts.force_topology=False

tree.render("%%inline",dpi=300,tree_style=ts,w=1200)

In [ ]:
import matplotlib.pyplot as plt
import logomaker as logomaker

logomaker.demo('fig1b')

In [ ]:
def PlotSequenceLogo(file)->None:
    
    alignment = AlignIO.read(file, "fasta")
    
    msa_dict={}
    for name,entry in entries.items():
        for aln in alignment:
            if entry in aln.id:
                new_numeration=list({i+1:L for i,L in enumerate(aln.seq) if L!='-'}.items())
                start,end=domains_dict[name]
                msa_dict[name]=[new_numeration[start-1][0],new_numeration[end-1][0]]

    chunks={}
    for seq in alignment:
        sequence_chunks=Chunk_list(str(seq.seq),100)
        for index,seq_fragment in enumerate(sequence_chunks):
            chunks.setdefault(index, []).append(seq_fragment)

    fig, axes = plt.subplots(nrows=len(chunks), ncols=1,figsize=(20,2*len(chunks)),sharex=False,sharey=False)         
    xtick_start=1

    yticks=np.arange(0,len(alignment)+1, 5)

    for index,seqs in chunks.items():
        ax=axes.ravel()[index]
        counts_mat=logomaker.alignment_to_matrix(sequences=seqs,to_type='counts',)

        logo=logomaker.Logo(counts_mat,shade_below=0.5,fade_below=0.5,ax=ax,color_scheme='black',fade_probabilities=True,
                            stack_order='big_on_top')

        logo.style_spines(visible=False)
        logo.style_xticks(rotation=90,fmt='%d', anchor=0,fontsize=10)
        logo.style_spines(spines=['left', 'bottom'], visible=True)
        ax.set_ylabel("Bits", labelpad=1,size=16)
        #ax.set_yticklabels(ax.get_yticks().astype(int),fontsize=10)

        xticklabels=ax.get_xticks()+xtick_start

        ax.set_xticklabels(xticklabels)
        ax.tick_params(width=1.5,size=2)


        colorsDomains={k:color for k,color in zip(msa_dict.keys(),sns.color_palette("Set1", len(msa_dict.keys())).as_hex())}

        for dname,msa_pointers in msa_dict.items():
            s=msa_pointers[0]
            e=msa_pointers[1]
            if s in xticklabels:
                plot_index=list(xticklabels).index(s)
                ax.axvline(x=ax.get_xticks()[plot_index], ymin=0, ymax=len(alignment)+1, color=colorsDomains[dname],linewidth=6,linestyle='--')
                ax.text(x=ax.get_xticks()[plot_index]+0.5, y=len(alignment),s=f'{dname}->',fontsize=12,horizontalalignment='left',fontweight='bold',color=colorsDomains[dname])
            if e in xticklabels:
                plot_index=list(xticklabels).index(e)
                ax.axvline(x=ax.get_xticks()[plot_index], ymin=0, ymax=len(alignment)+1, color=colorsDomains[dname],linewidth=6,linestyle='--')
                ax.text(x=ax.get_xticks()[plot_index]-0.5, y=len(alignment),s=f'<-{dname}',fontsize=12,horizontalalignment='right',fontweight='bold',color=colorsDomains[dname])

        xtick_start=xtick_start+ax.get_xticks()[-1]+1

    for ax in axes:
        ax.set_ylim(0,len(alignment)+1)
        ax.set_yticks(yticks,yticks)

    fig.subplots_adjust(bottom=0.0, top=1.0, left=0.1, right=0.8,wspace=0.2, hspace=0.4)
    
    if isinstance(outfile,str):
        fig.savefig(outfile,dpi=300,format='svg',bbox_inches='tight')

    else: pass

    return fig
    

In [ ]:
PlotSequenceLogo('ACCOACORAT.aln')

In [ ]:
import urllib.request as urlreq
from dash import html
import dash_bio as dashbio
from jupyter_dash import JupyterDash

In [ ]:
data = urlreq.urlopen('https://git.io/alignment_viewer_p53.fasta').read().decode('utf-8')

In [ ]:
with open('ACCOACORAT.aln','r') as f:
    data=f.read()

In [ ]:
chart=dashbio.AlignmentChart(id='alignment-viewer',data=data,colorscale='clustal',conservationcolorscale='Viridis',ticksteps=25,showid=False,showlabel=False,textsize=1,groupbars=True,overview='heatmap',width=1200)

In [ ]:
app = JupyterDash(__name__)
app.layout = html.Div(chart)
app.run_server(mode='inline',port=8030)